In [1]:
import tensorflow as tf

In [2]:
import numpy as np
from tensorflow.contrib import rnn
import random
import collections
import time
import itertools
from collections import Counter

In [3]:
#start_time = time.time()
training_file = 'fantasy.txt'

# logs_path = '/tensor_flow'
# writer = tf.summary.FileWriter(logs_path)

def read_data(fname):
    with open(fname) as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    content = [content[i].split() for i in range(len(content))]
    content = list(itertools.chain.from_iterable(content))
    content = np.array(content)
    #print(content[:20].shape)
    content = np.reshape(content, [-1, ])
    #print(content[:20].shape)
    return content

training_data = read_data(training_file)
print("Loaded training data...")

def build_dataset(words):
    #print(Counter(words))
    count = Counter(words).most_common()
    dictionary = dict()
    for word, count in count:
        dictionary[word] = len(dictionary)
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))

    return dictionary, reverse_dictionary

dictionary, reverse_dictionary = build_dataset(training_data)
vocab_size = len(dictionary)
print(vocab_size)

# Parameters
learning_rate = 0.001
training_iters = 50000
display_step = 1000
n_input = 3

# number of units in RNN cell
n_hidden = 512

# tf Graph input
x = tf.placeholder("float", [None, n_input, 1])
y = tf.placeholder("float", [None, vocab_size])

# RNN output node weights and biases
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, vocab_size]))
}
biases = {
    'out': tf.Variable(tf.random_normal([vocab_size]))
}

print(weights)
print(biases)
def RNN(x, weights, biases):

    # reshape to [1, n_input]
    x = tf.reshape(x, [-1, n_input])

    # Generate a n_input-element sequence of inputs
    # (eg. [had] [a] [general] -> [20] [6] [33])
    x = tf.split(x,n_input,1)
    #tf.Session.run(tf.global_variables_initializer())

    # 2-layer LSTM, each layer has n_hidden units.
    # Average Accuracy= 95.20% at 50k iter
    rnn_cell = rnn.MultiRNNCell([rnn.BasicLSTMCell(n_hidden),rnn.BasicLSTMCell(n_hidden)])

    # 1-layer LSTM with n_hidden units but with lower accuracy.
    # Average Accuracy= 90.60% 50k iter
    # Uncomment line below to test but comment out the 2-layer rnn.MultiRNNCell above
    #rnn_cell = rnn.BasicLSTMCell(n_hidden)

    # generate prediction
    outputs, states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)

    # there are n_input outputs but
    # we only want the last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

pred = RNN(x, weights, biases)

# Loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

# Model evaluation
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


Loaded training data...
244
{'out': <tf.Variable 'Variable:0' shape=(512, 244) dtype=float32_ref>}
{'out': <tf.Variable 'Variable_1:0' shape=(244,) dtype=float32_ref>}


In [4]:
logs_path = '/tensor_flow'
writer = tf.summary.FileWriter(logs_path)
# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as session:
    session.run(init)
    step = 0
    offset = random.randint(0,n_input+1)
    end_offset = n_input + 1
    acc_total = 0
    loss_total = 0

    writer.add_graph(session.graph)

    while step < training_iters:
        # Generate a minibatch. Add some randomness on selection process.
        if offset > (len(training_data)-end_offset):
            offset = random.randint(0, n_input+1)

        symbols_in_keys = [ [dictionary[ str(training_data[i])]] for i in range(offset, offset+n_input) ]
        symbols_in_keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])

        symbols_out_onehot = np.zeros([vocab_size], dtype=float)
        symbols_out_onehot[dictionary[str(training_data[offset+n_input])]] = 1.0
        symbols_out_onehot = np.reshape(symbols_out_onehot,[1,-1])

        _, acc, loss, onehot_pred = session.run([optimizer, accuracy, cost, pred], \
                                                feed_dict={x: symbols_in_keys, y: symbols_out_onehot})
        loss_total += loss
        acc_total += acc
        if (step+1) % display_step == 0:
            print("Iter= " + str(step+1) + ", Average Loss= " + \
                  "{:.6f}".format(loss_total/display_step) + ", Average Accuracy= " + \
                  "{:.2f}%".format(100*acc_total/display_step))
            acc_total = 0
            loss_total = 0
            symbols_in = [training_data[i] for i in range(offset, offset + n_input)]
            symbols_out = training_data[offset + n_input]
            symbols_out_pred = reverse_dictionary[int(tf.argmax(onehot_pred, 1).eval())]
            print("%s - [%s] vs [%s]" % (symbols_in,symbols_out,symbols_out_pred))
        step += 1
        offset += (n_input+1)
    print("Optimization Finished!")
    print("Elapsed time: ", elapsed(time.time() - start_time))
    print("Run on command line.")
    print("\ttensorboard --logdir=%s" % (logs_path))
    print("Point your web browser to: http://localhost:6006/")
    while True:
        prompt = "%s words: " % n_input
        sentence = input(prompt)
        sentence = sentence.strip()
        words = sentence.split(' ')
        if len(words) != n_input:
            continue
        try:
            symbols_in_keys = [dictionary[str(words[i])] for i in range(len(words))]
            for i in range(32):
                keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])
                onehot_pred = session.run(pred, feed_dict={x: keys})
                onehot_pred_index = int(tf.argmax(onehot_pred, 1).eval())
                sentence = "%s %s" % (sentence,reverse_dictionary[onehot_pred_index])
                symbols_in_keys = symbols_in_keys[1:]
                symbols_in_keys.append(onehot_pred_index)
            print(sentence)
        except:
            print("Word not in dictionary")


Iter= 1000, Average Loss= 4.736314, Average Accuracy= 11.80%
['an', 'astrologer', 'who'] - [is] vs [a]
Iter= 2000, Average Loss= 3.234104, Average Accuracy= 21.80%
['is', 'about', 'a'] - [pure] vs [politician.]
Iter= 3000, Average Loss= 3.253394, Average Accuracy= 25.60%
['a', 'principled', 'archer,'] - [a] vs [a]
Iter= 4000, Average Loss= 3.347451, Average Accuracy= 23.80%
['on', 'religion', 'is'] - [a] vs [story]
Iter= 5000, Average Loss= 2.919743, Average Accuracy= 25.60%
['a', 'skill,', 'climaxes'] - [with] vs [starts]
Iter= 6000, Average Loss= 2.760159, Average Accuracy= 29.60%
['strong', 'rascal.', 'It'] - [starts] vs [starts]
Iter= 7000, Average Loss= 2.399709, Average Accuracy= 36.30%
['noble,', 'and', 'a'] - [worried] vs [worried]
Iter= 8000, Average Loss= 2.568716, Average Accuracy= 34.90%
['a', 'bowman,', 'and'] - [an] vs [is]
Iter= 9000, Average Loss= 2.352784, Average Accuracy= 41.30%
['is', 'a', 'major'] - [part] vs [disaster]
Iter= 10000, Average Loss= 1.800269, Average 

NameError: name 'elapsed' is not defined

In [ ]:
print("Run on command line.")
print("\ttensorboard --logdir=%s" % (logs_path))
print("Point your web browser to: http://localhost:6006/")
while True:
    prompt = "%s words: " % n_input
    sentence = input(prompt)
    sentence = sentence.strip()
    words = sentence.split(' ')
    if len(words) != n_input:
        continue
    try:
        symbols_in_keys = [dictionary[str(words[i])] for i in range(len(words))]
        for i in range(32):
            keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])
            onehot_pred = session.run(pred, feed_dict={x: keys})
            onehot_pred_index = int(tf.argmax(onehot_pred, 1).eval())
            sentence = "%s %s" % (sentence,reverse_dictionary[onehot_pred_index])
            symbols_in_keys = symbols_in_keys[1:]
            symbols_in_keys.append(onehot_pred_index)
        print(sentence)
    except:
        print("Word not in dictionary")

Run on command line.
	tensorboard --logdir=/tensor_flow
Point your web browser to: http://localhost:6006/
3 words: The story begins
Word not in dictionary
3 words: The story is
Word not in dictionary
3 words: This is a
Word not in dictionary
3 words: The story is
Word not in dictionary
3 words: This is a
Word not in dictionary


In [ ]:
print(reverse_dictionary)

In [ ]:
print(reverse_dictionary)